<a href="https://colab.research.google.com/github/mahak2005/googlesolution/blob/main/GSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium
import pandas as pd
import numpy as np
import random
from datetime import date, timedelta
import folium
from folium.plugins import HeatMap
import time # variation in date generatio
NUM_RECORDS = 2000
START_DATE = date(2023, 1, 1)
END_DATE = date(2023, 12, 31)
DATE_RANGE_DAYS = (END_DATE - START_DATE).days
TAMIL_NADU_COASTAL_POINTS = [
    (13.08, 80.27, "Chennai"),
    (12.52, 80.19, "Mahabalipuram"),
    (11.94, 79.80, "Puducherry"),
    (11.74, 79.77, "Cuddalore"),
    (10.76, 79.84, "Nagapattinam"),
    (10.37, 79.13, "Thondi"),
    (9.28, 79.31, "Rameswaram"),
    (8.76, 78.11, "Thoothukudi"),
    (8.08, 77.53, "Kanyakumari")
]
SPECIES_LIST = ['Sardine', 'Mackerel', 'Anchovy', 'Tuna', 'Snapper', 'Grouper', 'Prawns', 'Crabs', 'Seer Fish', 'Pomfret']
def generate_fish_data(num_records):
    data = []
    for i in range(num_records):
        random_days = random.randint(0, DATE_RANGE_DAYS)
        record_date = START_DATE + timedelta(days=random_days)
        species = random.choice(SPECIES_LIST)
        base_lat, base_lon, _ = random.choice(TAMIL_NADU_COASTAL_POINTS)
        lat_noise = random.uniform(-0.15, 0.15)
        lon_noise = random.uniform(-0.15, 0.15)
        latitude = round(base_lat + lat_noise, 6)
        longitude = round(base_lon + lon_noise, 6)
        catch_weight = round(np.random.gamma(shape=2.5, scale=10) + 1.0, 2)
        if catch_weight > 150: catch_weight = 150
        depth = round(random.uniform(5, 150), 1)
        wind_speed = round(random.uniform(0, 35), 1)
        temperature = round(random.uniform(24.0, 36.0), 1)
        rain_mm = round(max(0, np.random.normal(loc=5, scale=15)), 1) if random.random() < 0.2 else 0.0
        data.append({
            "Date": record_date, "Species": species, "Latitude": latitude,
            "Longitude": longitude, "Catch Weight (kg)": catch_weight,
            "Depth (m)": depth, "Wind Speed (knots)": wind_speed,
            "Temperature (°C)": temperature, "Rain (mm)": rain_mm
        })
    return pd.DataFrame(data)
print(f"Generating {NUM_RECORDS} synthetic fish catch records...")
df_fish = generate_fish_data(NUM_RECORDS)
print("Dataset generation complete.")
print("\nFull Dataset Sample:")
print(df_fish.head())

Generating 2000 synthetic fish catch records...
Dataset generation complete.

Full Dataset Sample:
         Date  Species   Latitude  Longitude  Catch Weight (kg)  Depth (m)  \
0  2023-10-14  Sardine   8.723934  78.212845              20.20        9.9   
1  2023-02-25   Prawns  10.322455  79.260908              27.64       19.8   
2  2023-03-05  Grouper  11.988081  79.915074               6.20       66.4   
3  2023-08-11   Prawns  11.874895  79.777507              12.16       78.9   
4  2023-04-02  Sardine  11.925541  79.802126              19.16      128.1   

   Wind Speed (knots)  Temperature (°C)  Rain (mm)  
0                10.8              28.6        0.0  
1                 3.9              29.7        0.0  
2                25.5              25.0        0.0  
3                13.8              27.6        0.0  
4                18.5              27.0        0.0  


In [ ]:

df_fish.to_csv("fish_catch_data.csv", index=False)
print("CSV saved as 'fish_catch_data.csv'")


CSV saved as 'fish_catch_data.csv'


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


df = pd.read_csv("fish_catch_data.csv")

df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek


le_species = LabelEncoder()
df['Species_Encoded'] = le_species.fit_transform(df['Species'])
df = df.drop(columns=['Date', 'Species'])

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df)
df_scaled = pd.DataFrame(scaled_features, columns=df.columns)

X = df_scaled.drop(columns=['Catch Weight (kg)'])
y = df_scaled['Catch Weight (kg)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

catch_model = RandomForestRegressor(n_estimators=100, random_state=42)
catch_model.fit(X_train, y_train)

y_pred = catch_model.predict(X_test)

print("Fish Catch Prediction")
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np




Fish Catch Prediction
MSE: 1.101713805589643
R² Score: -0.10495860578476734


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Predict
y_pred = catch_model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n🎣 Fish Catch Prediction Model Metrics:")
print("MAE:", round(mae, 3))
print("MSE:", round(mse, 3))
print("RMSE:", round(rmse, 3))
print("R² Score:", round(r2, 3))


🎣 Fish Catch Prediction Model Metrics:
MAE: 0.83
MSE: 1.102
RMSE: 1.05
R² Score: -0.105


In [ ]:

X_weather = df_scaled.drop(columns=['Temperature (°C)', 'Rain (mm)'])
y_temp = df_scaled['Temperature (°C)']
y_rain = df_scaled['Rain (mm)']


X_train_w, X_test_w, y_train_temp, y_test_temp = train_test_split(X_weather, y_temp, test_size=0.2, random_state=42)
_, _, y_train_rain, y_test_rain = train_test_split(X_weather, y_rain, test_size=0.2, random_state=42)

temp_model = RandomForestRegressor(random_state=42)
temp_model.fit(X_train_w, y_train_temp)

rain_model = RandomForestRegressor(random_state=42)
rain_model.fit(X_train_w, y_train_rain)

temp_pred = temp_model.predict(X_test_w)
rain_pred = rain_model.predict(X_test_w)


print("\nWeather Forecasting")
print("Temperature R² Score:", r2_score(y_test_temp, temp_pred))
print("Rainfall R² Score:", r2_score(y_test_rain, rain_pred))



Weather Forecasting
Temperature R² Score: -0.06838159595715965
Rainfall R² Score: -0.10068724620021485


In [ ]:
# Temperature Accuracy
temp_pred = temp_model.predict(X_test_w)
temp_mse = mean_squared_error(y_test_temp, temp_pred)
temp_rmse = np.sqrt(temp_mse)
temp_mae = mean_absolute_error(y_test_temp, temp_pred)
temp_r2 = r2_score(y_test_temp, temp_pred)

print("\n🌡️ Temperature Prediction Metrics:")
print("MAE:", round(temp_mae, 3))
print("MSE:", round(temp_mse, 3))
print("RMSE:", round(temp_rmse, 3))
print("R² Score:", round(temp_r2, 3))

# Rainfall Accuracy
rain_pred = rain_model.predict(X_test_w)
rain_mse = mean_squared_error(y_test_rain, rain_pred)
rain_rmse = np.sqrt(rain_mse)
rain_mae = mean_absolute_error(y_test_rain, rain_pred)
rain_r2 = r2_score(y_test_rain, rain_pred)

print("\n🌧️ Rainfall Prediction Metrics:")
print("MAE:", round(rain_mae, 3))
print("MSE:", round(rain_mse, 3))
print("RMSE:", round(rain_rmse, 3))
print("R² Score:", round(rain_r2, 3))



🌡️ Temperature Prediction Metrics:
MAE: 0.895
MSE: 1.105
RMSE: 1.051
R² Score: -0.068

🌧️ Rainfall Prediction Metrics:
MAE: 0.625
MSE: 1.341
RMSE: 1.158
R² Score: -0.101


In [ ]:
print("\n--- Species Selection ---")

available_species_in_data = df_fish['Species'].unique()



species_options_str = ", ".join(SPECIES_LIST)
prompt_message = (
    f"Enter the name of the fish species to visualize.\n"
    f"Available options are: {species_options_str}\n"
    f"Your choice: "
)
selected_species = input(prompt_message).strip().title()




if selected_species in SPECIES_LIST:
    print(f"\nFiltering data for species: {selected_species}")

    filtered_df = df_fish[df_fish['Species'] == selected_species].copy()


    if filtered_df.empty:
        print(f"Note: '{selected_species}' is a valid species, but no catch records were generated for it in this random dataset.")
        heatmap_layer_name = f'No Data Generated for {selected_species}'
    else:
        print(f"Found {len(filtered_df)} records for {selected_species}.")
        print("\nSample Filtered Data:")
        print(filtered_df.head())
        heatmap_layer_name = f'{selected_species} Catch Intensity'

else:

    print(f"\nWarning: Species '{selected_species}' is not in the list of recognized species ({species_options_str}).")
    print("Displaying heatmap for ALL species instead.")

    filtered_df = df_fish
    heatmap_layer_name = 'All Species Catch Intensity (Invalid Input)'
    if filtered_df.empty:
         print("The generated dataset is empty.")




print("\nGenerating Heatmap...")

#map center
center_lat = np.mean([p[0] for p in TAMIL_NADU_COASTAL_POINTS])
center_lon = np.mean([p[1] for p in TAMIL_NADU_COASTAL_POINTS])


fish_species_heatmap = folium.Map(location=[center_lat, center_lon], zoom_start=7, tiles='OpenStreetMap')


if not filtered_df.empty:
    heat_data = [[row['Latitude'], row['Longitude'], row['Catch Weight (kg)']] for index, row in filtered_df.iterrows()]


    HeatMap(heat_data,
            name=heatmap_layer_name,
            min_opacity=0.3,
            radius=12,
            blur=10,
           ).add_to(fish_species_heatmap)
else:
    print("Skipping HeatMap layer generation as there is no data to display.")



for lat, lon, name in TAMIL_NADU_COASTAL_POINTS:
    folium.Marker(
        location=[lat, lon],
        popup=f"<i>{name}</i>",
        tooltip=name,
        icon=folium.Icon(color='gray', icon='anchor', prefix='fa')
    ).add_to(fish_species_heatmap)


folium.LayerControl().add_to(fish_species_heatmap)

print("Heatmap generation complete. Displaying map...")


fish_species_heatmap


species_filename = selected_species.replace(' ', '_') if selected_species in SPECIES_LIST else 'all_species'
file_path = f"/content/tamil_nadu_{species_filename}_heatmap.html"
fish_species_heatmap.save(file_path)
print(f"\nMap saved to {file_path}")


--- Species Selection ---
Enter the name of the fish species to visualize.
Available options are: Sardine, Mackerel, Anchovy, Tuna, Snapper, Grouper, Prawns, Crabs, Seer Fish, Pomfret
Your choice: Sardine

Filtering data for species: Sardine
Found 185 records for Sardine.

Sample Filtered Data:
          Date  Species   Latitude  Longitude  Catch Weight (kg)  Depth (m)  \
24  2023-09-24  Sardine   8.097525  77.638861              17.94       13.8   
26  2023-02-07  Sardine  10.767097  79.844252              20.67       67.0   
28  2023-10-27  Sardine   9.332435  79.451742               9.52       78.4   
42  2023-03-22  Sardine   8.186178  77.622203              27.77      130.0   
43  2023-02-02  Sardine   8.714974  78.208064              25.48      130.9   

    Wind Speed (knots)  Temperature (°C)  Rain (mm)  
24                 1.3              26.4        0.0  
26                33.1              35.9        0.0  
28                19.3              25.8        0.0  
42          